##   Computer Aided Detection(CAD) for the Diagnosis in SKIN Images
* By Abdullah Thabit and Tewodros W. Arega

In [ ]:
import cv2
import numpy as np
import json
from imutils import paths

import matplotlib.pyplot as plt
import skimage.filters
import scipy.stats

import scipy.ndimage.morphology
import sys
import os
import numpy as np
import cv2
import progressbar
import matplotlib.pyplot as plt
import SimpleITK as sitk
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.transform import match_histograms
from sklearn.metrics import classification_report
from scipy import ndimage
import imutils
import mpld3
import random
import numpy.ma as ma

import mahotas as mt
from skimage.feature import hog
from scipy.stats import skew, kurtosis
from scipy import ndimage as ndi
from skimage import feature

from skimage import data
from skimage.filters import gabor_kernel

from ipywidgets import IntProgress
from IPython.display import display
import time
from progress.bar import Bar


## Image Path

In [ ]:
#Specify the training and the testing image paths
TRAIN_PATH = 'D:/MAIA/MAIA-3rd Semester UdG/CAD/MiniProject-I Classical/Dermoscopy/train'
VAL_PATH = 'D:/MAIA/MAIA-3rd Semester UdG/CAD/MiniProject-I Classical/Dermoscopy/test'

## Feature Extraction

In [ ]:
'''
A class that computes the the Local Binary Pattern and returns the normalized histogram

''' 
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
 
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return hist  

'''
    Color feature
     - Computes the color stastics of RGB, HSV and LAB color spaces of the image:
        It calculates the mean, std, skew and kurtosis of each color spaces
        
        Inputs:
            img_RGB: RGB image
        Returns:
            the color stastics feature vector
'''  

def meanstd_fun(image):
    # mean and std
    img_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)    
    img_LAB = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)    

    mean_i, std_i = cv2.meanStdDev(image)
    mean_hsv, std_hsv = cv2.meanStdDev(img_HSV) 
    mean_lab, std_lab = cv2.meanStdDev(img_LAB) 

    skw = np.array([ skew(image[:,:,0].flatten()), skew(image[:,:,1].flatten()), skew(image[:,:,2].flatten()), 
                    kurtosis(image[:,:,0].flatten()), kurtosis(image[:,:,1].flatten()), kurtosis(image[:,:,2].flatten()),
                   skew(img_HSV[:,:,0].flatten()), skew(img_HSV[:,:,1].flatten()), skew(img_HSV[:,:,2].flatten()), 
                    kurtosis(img_HSV[:,:,0].flatten()), kurtosis(img_HSV[:,:,1].flatten()), kurtosis(img_HSV[:,:,2].flatten()),
                    skew(img_LAB[:,:,0].flatten()), skew(img_LAB[:,:,1].flatten()), skew(img_LAB[:,:,2].flatten()), 
                    kurtosis(img_LAB[:,:,0].flatten()), kurtosis(img_LAB[:,:,1].flatten()), kurtosis(img_LAB[:,:,2].flatten()) ])
    skw = np.reshape(skw,(-1,1))

    meanstd = np.concatenate((mean_i, std_i, mean_hsv, std_hsv, mean_lab, std_lab, skw), axis=0)            
    
    return meanstd

'''
 Color feature
  - Compute and concatenate the color histogram of RGB, HSV and LAB color spaces of the image:        
        
        Inputs:
            img: RGB image
        Returns:
            the color histogram feature vector
'''
def color_hist_fun(image):
        
    
    img_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    img_LAB = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)    

    color_hist = cv2.calcHist([img_HSV], [0, 1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    color_hist_LAB = cv2.calcHist([img_LAB], [0, 1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    color_hist_RGB = cv2.calcHist([image], [0, 1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    color_hist = color_hist.flatten()
    color_hist_RGB = color_hist_RGB.flatten()
    color_hist_LAB = color_hist_LAB.flatten()
    color_hist = np.expand_dims(color_hist, axis=-1)
    color_hist_RGB = np.expand_dims(color_hist_RGB, axis=-1)
    color_hist_LAB = np.expand_dims(color_hist_LAB, axis=-1)
    color_hist = np.concatenate((color_hist, color_hist_LAB, color_hist_RGB), axis = 0)
    
    return color_hist

'''
Texture feature
 Computes Gray-Level Co-Occurrence Matrix (GLCM) 
     and stastics from GLCM like contrast, correlation, energy, homogeneity and etc of a grayscale image:        
     It uses mahotas's haralick glcm.
        
        Inputs:
            img_gray: grayscale image
        Returns:
            the glcm stastics feature vector
'''
def glcm_fun(img_gray):
    
    glcm = mt.features.haralick(img_gray)    
    glcm_f = glcm.mean(axis=0)
    glcm_f = np.expand_dims(glcm_f,axis=-1)    
    
    return glcm_f

'''
Texture feature
 Computes the histogram of local binary pattern of a grayscale image:              
        Inputs:
            img_gray: grayscale image
            no_points: the number of points in a circularly symmetric neighborhood
            radius: radius of the circular neighborhood
        Returns:
            the lbp feature vector
'''  
def lbp_fun(img_gray): 
    
    no_points = 8
    radius = 3
    descr = LocalBinaryPatterns(no_points, radius)        
    hist = descr.describe(img_gray)
    hist = np.expand_dims(hist, axis=-1)
    
    return hist
'''
Shape feature
 Computes the hu moments of a binary image:              
        Inputs:
            segmented_image: binary image            
        Returns:
            the concatenation of the seven hu moments
'''
def humoments_fun(segmented_image):
    
    moments = cv2.moments(segmented_image)

    # Calculate Hu Moments
    huMoments = cv2.HuMoments(moments)

    # Log transform of hu moments b/se they have large ranges
    for i in range(0,7):
        huMoments[i] = -1* np.copysign(1.0, huMoments[i]) * np.log10(abs(huMoments[i])+1e-30)        
    
    return huMoments  

'''
Texture feature
 Computes the mean, std, local energy and mean ampitude of filtered image using gabor kernel:        
       
        Inputs:
            image: grayscale image
            kernels: gabor filter kernel bank
        Returns:
            the gabor feature vector
''' 
def gabor_feature_fun(image, kernels):
    
    #extracts mean and std from the filtered images
    #local energy and mean ampitude
    feats = np.zeros((len(kernels), 4), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')        
        feats[k, 0] = filtered.mean()
        feats[k, 1] = filtered.var()
        feats[k, 2] = np.sum(filtered**2) #energy
        feats[k, 3] = np.sum(np.abs(filtered)) #mean ampitude
    gabor_feats = np.concatenate((feats[:, 0].flatten(), feats[:, 1].flatten(),
                                  feats[:, 2].flatten(), feats[:, 3].flatten()),axis=0)
    
    return gabor_feats
'''
Texture feature
 Computes the histogram of gradients(HOG) of grayscale image:        
       
        Inputs:
            img_gray: grayscale image            
        Returns:
            the HOG feature vector
'''             
def hog_fun(img_gray):
    
    hog_f, hog_image = hog(img_gray, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=False)             
    hog_f = np.expand_dims(hog_f, axis=-1)
    
    return hog_f

## Sort filenames for testing

In [ ]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
############################################
# Prepare gabor filter bank kernels
# with orientation of 0, 1/4*pi, 1/2*pi, 3/4*pi, pi and frequency of 0.05 and 0.25,
# sigma of 1 and 3
############################################

gabor_kernels = []
for theta in range(4):
    theta = theta / 4. * np.pi
    for sigma in (1, 3):
        for frequency in (0.05, 0.25):
            g_kernel = np.real(gabor_kernel(frequency, theta=theta,
                                          sigma_x=sigma, sigma_y=sigma))
            gabor_kernels.append(g_kernel)

## Sliding Window

In [ ]:
'''
  Divides the image into equal size regions

    Inputs:
        image: the image to be divided
        stepSize: the number of pixels to “skip” while sliding
        windowSize: the width and height of the window we are going to extract from the image
    Returns:
        the extracted window(region) from the image
'''
def sliding_window(image,  windowSize, no_channels):
    # slide a window across the image, image.shape[1] = 600, image.shape[0] = 450, stepsize_x = windosize[1]  
    stepSize_x  = windowSize[1]
    stepSize_y = windowSize[0]
    for x in range(0, image.shape[0], stepSize_x):
        for y in range(0, image.shape[1], stepSize_y):
            # yield the current window
            if no_channels == 1:                
                yield (x, y, image[x:x + windowSize[1], y:y + windowSize[0]])
            else:
                yield (x, y, image[x:x + windowSize[1], y:y + windowSize[0],:])

## Segmentation Function

In [ ]:
'''
Hair removal and Segmentation of Skin Lesions:
    Input:
        img_clahe: contrast enhanced grayscale image
        gray: grayscale image with out contrast enhancment
        imageRGB: RGB skin image
    Returns:
        finalwatershed1: segmented mask
        inpaintedWhite: grayscale image after hair removal
        inpaintedRGB: RGB skin image after hair removal
'''
def segment_skin(img_clahe, gray, imageRGB):
    
    ## Hair Removal
    
    kernelblackhat = cv2.getStructuringElement(1, (17, 17))
    blackhat = cv2.morphologyEx(image_clahe, cv2.MORPH_BLACKHAT, kernelblackhat)
    ret, threshblackhat = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    
    inpaintedWhite = cv2.inpaint(image_clahe, threshblackhat, 1, cv2.INPAINT_TELEA)
    inpaintedRGB = cv2.inpaint(imageRGB, threshblackhat, 1, cv2.INPAINT_TELEA)   
            
    inpaintedWhite = clahe.apply(inpaintedWhite) 
    
    ## Apply Ellipse Mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mymask = np.zeros_like(inpaintedWhite) 
    ellipse_mask = cv2.ellipse(mymask, (300,225),(300,225), 0, 0, 360, 1, -1)
    (_, thresh) = cv2.threshold(inpaintedWhite, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    thresh = thresh * ellipse_mask
    kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel3)
    
    # Seed selection using distance transform
    D = ndimage.distance_transform_edt(thresh)
    localMax = peak_local_max(D, indices=False, min_distance=20,  labels=thresh,
                              num_peaks_per_label = 10)

    # creating marker or seed from localmax
    markers = ndimage.label(localMax)[0]
    markers = np.uint16(markers)

    # Mask the seeds in the corners 
    markers[:100,:] = 0 
    markers[:,500:] = 0                
    markers[:,:100] = 0
    markers[350:,:] = 0
    
    # Watershed Segmentation and Post Processing
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    label_watershed = watershed(-D, markers, mask=thresh) 
    final_watershed = (label_watershed > 0) * 1
    final_watershed = final_watershed.astype(np.uint8)
    final_watershed = cv2.morphologyEx(final_watershed, cv2.MORPH_DILATE, kernel2, iterations = 2)
    final_watershed1 = final_watershed
    final_watershed1 = final_watershed1.astype(np.uint8)
    final_watershed1 = cv2.morphologyEx(final_watershed1, cv2.MORPH_DILATE, kernel2, iterations = 2) # to view markers
    ########################################################
    # Filling Holes
    ########################################################
    contour,hier  = cv2.findContours(final_watershed1,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contour:            
        cv2.drawContours(final_watershed1,[cnt],0,255,-1)    
        
    return final_watershed1, inpaintedWhite, inpaintedRGB


## Set the window size and number of regions for sliding window

In [ ]:
%matplotlib inline
from sklearn import svm

total_no = 4800

#Sliding window size
resize = 1
winW = np.int(600/resize)
winH = np.int(450/resize)


In [ ]:

dirs_all = os.listdir(TRAIN_PATH)
dirs_all.sort()

humoments_feature = []
lbp_feature = []
glcm_feature = []
labels = []
meanstd_feature = []
gabor_feature = []
color_hist_feature = []
hog_feature = []

max_count = 6000

# set up the progress bar
widgets = ["Reading: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=max_count, widgets=widgets).start()
print("[INFO] extracting training features...")
q = 0

# Iterate over the training images
for n_type, typefile in enumerate(dirs_all):

    lesiontype = os.listdir(os.path.join(TRAIN_PATH, typefile))
    if n_type == 0:
        print("Lesion")
    else:
        print("Benign")

    for n, filename in enumerate(lesiontype):
        imgpath = os.path.join(TRAIN_PATH, typefile, filename)
        image = cv2.imread(imgpath)
        image = cv2.GaussianBlur(image, (7, 7), 0)
        
        if n_type == 0: # les = 1
            labels.append(1)
        else:          # nev = 0
            labels.append(0)

        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        #Contrast Enhancment
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        image_clahe = clahe.apply(gray)
        
        ###################################################
        # Segmentation of skin image
        ##################################################
        # final_watershed1: segmented mask
        # inpaintedWhite: grayscale skin image after hair removal
        # inpaintedRGB : RGB skin image after hair removal
        
        final_watershed1, inpaintedWhite, inpaintedRGB = segment_skin(image_clahe, gray,image)        
                
        '''
            Feature Extraction
        '''
        
        ########################################################
        # Hu moments - shape descriptor
        ########################################################        
        
        huMoments = humoments_fun(final_watershed1)
        
        if n_type == 0 and n == 0:
            humoments_feature = huMoments                        
        else:
            humoments_feature = np.concatenate((humoments_feature, huMoments), axis=1)            
        
        
        ###########################################
        # Mean, std, skewness and kurtosis of each channel
        ###########################################        
        
        # Sliding window        
        meanstd_window = []
        for (x, y, window) in sliding_window(inpaintedRGB, windowSize=(winW, winH), no_channels =3):
            # extract features from the windows and concatinate them
            meanstd = meanstd_fun(window)
            if x == 0 and y == 0:
                meanstd_window = meanstd
            else:
                meanstd_window = np.concatenate((meanstd_window, meanstd),axis=0) 
                        
        if n_type == 0 and n == 0:
            meanstd_feature = meanstd_window
        else:
            meanstd_feature = np.concatenate((meanstd_feature, meanstd_window),axis=1)                     
        
       ###########################################
        # Color Histogram of RGB, HSV and LAB
        ###########################################                   
            
        # Sliding window        
        color_hist_window = []
        for (x, y, window) in sliding_window(inpaintedRGB, windowSize=(winW, winH),no_channels =3):
            # extract features from the windows and concatinate them
            color_hist = color_hist_fun(window)
            if x == 0 and y == 0:
                color_hist_window = color_hist
            else:
                color_hist_window = np.concatenate((color_hist_window, color_hist),axis=0) 
                
        if n_type == 0 and n == 0:
            color_hist_feature = color_hist_window
        else:
            color_hist_feature = np.concatenate((color_hist_feature, color_hist_window), axis=1) 
        
        ###########################################
        # LBP features - texture descriptor
        ###########################################
        
        # Sliding window        
        hist_window = []
        for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
            # extract features from the windows and concatinate them
            
            hist = lbp_fun(window)
            if x == 0 and y == 0:
                hist_window = hist
            else:
                hist_window = np.concatenate((hist_window, hist),axis=0)
                
        if n_type == 0 and n == 0:
            lbp_feature = hist_window
        else:
            lbp_feature = np.concatenate((lbp_feature, hist_window),axis=1) 
            
            
        ###########################################
        # GLCM based statistical descriptors - texture descriptor        
        ###########################################
        # calculate haralick texture features for 4 types of adjacency
        
        # Sliding window        
        glcm_f_window = []
        for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
            # extract features from the windows and concatinate them
            
            glcm_f = glcm_fun(window)
            if x == 0 and y == 0:
                glcm_f_window = glcm_f
            else:
                glcm_f_window = np.concatenate((glcm_f_window, glcm_f),axis=0)
                
        if n_type == 0 and n == 0:
            glcm_feature = glcm_f_window
        else:
            glcm_feature = np.concatenate((glcm_feature, glcm_f_window), axis=1)     
        
        ###########################################
        # Gabor features - texture descriptor
        ###########################################        
        
        # Sliding window        
        gabor_window = []
        for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
            # extract features from the windows and concatinate them
            
            gabor_f = gabor_feature_fun(window, gabor_kernels)
            if x == 0 and y == 0:
                gabor_window = gabor_f
            else:
                gabor_window = np.concatenate((gabor_window, gabor_f),axis=0)
        gabor_window = np.expand_dims(gabor_window, axis=-1)        
        
        if n_type == 0 and n == 0:
            gabor_feature = gabor_window
        else:
            gabor_feature = np.concatenate((gabor_feature, gabor_window),axis=1) 
        
        ###################################################
        # HOG features - structure and texture descriptor
        ###################################################
        # Sliding window        
        hog_window = []
        for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
            # extract features from the windows and concatinate them
            
            hog_f = hog_fun(window)
            if x == 0 and y == 0:
                hog_window = hog_f
            else:
                hog_window = np.concatenate((hog_window, hog_f),axis=0)
        hog_window = np.expand_dims(hog_window, axis=-1)                        
            
        if n_type == 0 and n == 0:
            hog_feature = hog_window
        else:
            hog_feature = np.concatenate((hog_feature, hog_window), axis=1)                           
        
        #Progress Bar
        q += 1
        pbar.update(q) 

# Transpose so that the feature vector has (#no_samples,#no_features) shape        
humoments_feature = np.transpose(humoments_feature)
lbp_feature = np.transpose(lbp_feature)
meanstd_feature = np.transpose(meanstd_feature)
glcm_feature = np.transpose(glcm_feature)
gabor_feature = np.transpose(gabor_feature)
color_hist_feature = np.transpose(color_hist_feature)
hog_feature = np.transpose(hog_feature)

pbar.finish()
print("Done")

## Extract features from testing images

In [ ]:
dirs_all = os.listdir(VAL_PATH)
dirs_all.sort(key=natural_keys)

max_count = 1015

humoments_feature_val = []
meanstd_feature_val = []
lbp_feature_val = []
glcm_feature_val = []
gabor_feature_val = []
color_hist_feature_val = []
hog_feature_val = []
labels_val = []

# set up the progress bar
widgets = ["Reading: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=max_count, widgets=widgets).start()
print("[INFO] extracting validation features...")
q = 0

#Iterate over the testing images
for n_type, typefile in enumerate(dirs_all):
    
    #Read Image
    imgpath = os.path.join(VAL_PATH, typefile)
    image = cv2.imread(imgpath)
    
    #Gaussian Blurring
    image = cv2.GaussianBlur(image, (7, 7), 0)    

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Contrast Enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image_clahe = clahe.apply(gray)

    ###################################################
    # Segmentation
    ##################################################
    # final_watershed1: segmented mask
    # inpaintedWhite: grayscale skin image after hair removal
    # inpaintedRGB : RGB skin image after hair removal

    final_watershed1,inpaintedWhite, inpaintedRGB = segment_skin(image_clahe, gray, image)


    '''
        Feature Extraction
    '''
    
    ########################################################
    # Hu moments - shape descriptor
    ########################################################

    huMoments = humoments_fun(final_watershed1)
        
    if n_type == 0:
        humoments_feature_val = huMoments                        
    else:
        humoments_feature_val = np.concatenate((humoments_feature_val, huMoments), axis=1)           
    
    ########################################
    # Mean, std, skewness and kurtosis of each channel    
    #########################################
    # Sliding window        
    meanstd_window = []
    for (x, y, window) in sliding_window(inpaintedRGB, windowSize=(winW, winH),no_channels =3):
        # extract features from the windows and concatinate them
        meanstd = meanstd_fun(window)
        if x == 0 and y == 0:
            meanstd_window = meanstd
        else:
            meanstd_window = np.concatenate((meanstd_window, meanstd),axis=0) 

    if n_type == 0 :
        meanstd_feature_val = meanstd_window
    else:
        meanstd_feature_val = np.concatenate((meanstd_feature_val, meanstd_window),axis=1)

    ###########################################
    # Color Histogram of RGB, HSV and LAB color spaces
    ###########################################

    color_hist_window = []
    for (x, y, window) in sliding_window(inpaintedRGB, windowSize=(winW, winH),no_channels =3):
        # extract features from the windows and concatinate them
        color_hist = color_hist_fun(window)
        if x == 0 and y == 0:
            color_hist_window = color_hist
        else:
            color_hist_window = np.concatenate((color_hist_window, color_hist),axis=0)                 

    if n_type == 0 :
        color_hist_feature_val = color_hist_window
    else:
        color_hist_feature_val = np.concatenate((color_hist_feature_val, color_hist_window), axis=1) 

    ###########################################
    # LBP features - texture descriptor
    ###########################################
    # Sliding window        
    hist_window = []
    for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
        # extract features from the windows and concatinate them

        hist = lbp_fun(window)
        if x == 0 and y == 0:
            hist_window = hist
        else:
            hist_window = np.concatenate((hist_window, hist),axis=0)

    if n_type == 0 :
        lbp_feature_val = hist_window
    else:
        lbp_feature_val = np.concatenate((lbp_feature_val,hist_window),axis=1) 


    ###########################################
    # GLCM based statistical descriptors - texture descriptor    
    ###########################################
    # calculate haralick texture features for 4 types of adjacency
    # Sliding window        
    glcm_f_window = []
    for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
        # extract features from the windows and concatinate them

        glcm_f = glcm_fun(window)
        if x == 0 and y == 0:
            glcm_f_window = glcm_f
        else:
            glcm_f_window = np.concatenate((glcm_f_window, glcm_f),axis=0)

    if n_type == 0:
        glcm_feature_val = glcm_f_window
    else:
        glcm_feature_val = np.concatenate((glcm_feature_val, glcm_f_window), axis=1)    
                                
    ###########################################
    # Gabor features - texture descriptor
    ###########################################        

    # Sliding window        
    gabor_window = []
    for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
        # extract features from the windows and concatinate them

        gabor_f = gabor_feature_fun(window, gabor_kernels)
        if x == 0 and y == 0:
            gabor_window = gabor_f
        else:
            gabor_window = np.concatenate((gabor_window, gabor_f),axis=0)
    gabor_window = np.expand_dims(gabor_window, axis=-1)        

    if n_type == 0 :
        gabor_feature_val = gabor_window
    else:
        gabor_feature_val = np.concatenate((gabor_feature_val, gabor_window),axis=1) 


    ##################################################
    # HOG features - structure and texture descriptor
    ####################################################
    hog_window = []
    for (x, y, window) in sliding_window(inpaintedWhite, windowSize=(winW, winH),no_channels =1):
        # extract features from the windows and concatinate them

        hog_f = hog_fun(window)
        if x == 0 and y == 0:
            hog_window = hog_f
        else:
            hog_window = np.concatenate((hog_window, hog_f),axis=0)
    hog_window = np.expand_dims(hog_window, axis=-1)

    if n_type == 0:
        hog_feature_val = hog_window
    else:
        hog_feature_val = np.concatenate((hog_feature_val, hog_window), axis=1)        
    
    #Progress Bar
    q += 1
    pbar.update(q) 

# Transpose so that the feature vector has (#no_samples,#no_features) shape    
humoments_feature_val = np.transpose(humoments_feature_val)
meanstd_feature_val = np.transpose(meanstd_feature_val)
lbp_feature_val = np.transpose(lbp_feature_val)
glcm_feature_val = np.transpose(glcm_feature_val)
gabor_feature_val = np.transpose(gabor_feature_val)
color_hist_feature_val = np.transpose(color_hist_feature_val)
hog_feature_val = np.transpose(hog_feature_val)

pbar.finish()
print("Done")

In [ ]:
humoments.shape

## Normalization of Features using mean and std

In [ ]:
#Calculate the mean and std of each features
mean_meanstd = np.mean(meanstd_feature, axis=0)
std_meanstd = np.std(meanstd_feature, axis=0) + 1e-5
mean_lbp = np.mean(lbp_feature, axis=0)
std_lbp = np.std(lbp_feature, axis=0)  + 1e-5
mean_glcm = np.mean(glcm_feature, axis=0)
std_glcm = np.std(glcm_feature, axis=0)  + 1e-5
mean_colorhist = np.mean(color_hist_feature, axis=0)
std_colorhist = np.std(color_hist_feature, axis=0)  + 1e-5
mean_humoments = np.mean(humoments_feature, axis=0)
std_humoments = np.std(humoments_feature, axis=0)  + 1e-5
mean_gabor = np.mean(gabor_feature, axis=0)
std_gabor = np.std(gabor_feature, axis=0)  + 1e-5
mean_hog = np.mean(hog_feature, axis=0)
std_hog = np.std(hog_feature, axis=0)  + 1e-5

# Normalize each training features
meanstd_feature_N = (meanstd_feature - mean_meanstd) / std_meanstd
lbp_feature_N = (lbp_feature - mean_lbp) / std_lbp
glcm_feature_N = (glcm_feature - mean_glcm) / std_glcm
color_hist_feature_N = (color_hist_feature - mean_colorhist) / std_colorhist
humoments_feature_N = (humoments_feature - mean_humoments) / std_humoments
gabor_feature_N = (gabor_feature - mean_gabor) / std_gabor
hog_feature_N = (hog_feature - mean_hog) / std_hog

# Normalize each test features
meanstd_feature_val_N = (meanstd_feature_val - mean_meanstd) / std_meanstd
lbp_feature_val_N = (lbp_feature_val - mean_lbp) / std_lbp
glcm_feature_val_N = (glcm_feature_val - mean_glcm) / std_glcm
color_hist_feature_val_N = (color_hist_feature_val - mean_colorhist) / std_colorhist
humoments_feature_val_N = (humoments_feature_val - mean_humoments) / std_humoments
gabor_feature_val_N = (gabor_feature_val - mean_gabor) / std_gabor
hog_feature_val_N = (hog_feature_val - mean_hog) / std_hog

## Combine the normalized features

In [ ]:
all_features = np.concatenate((lbp_feature_N, meanstd_feature_N, glcm_feature_N),axis=1) 
all_features.shape

In [ ]:
all_feature_val = np.concatenate((lbp_feature_val_N, meanstd_feature_val_N, glcm_feature_val_N), axis=1) 
all_feature_val.shape

## Training the features using Different classifiers

In [ ]:
'''
'linear_svm' for linear svm
'svm' for svm with rbf kernel
'bagging' for bagging classifier
'randomforest' for random forest

'''
classifier_type = 'svm'

In [ ]:
%%time
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

cart = DecisionTreeClassifier(max_depth=5, class_weight="balanced")

if classifier_type == 'linear_svm':
    model = svm.LinearSVC(C=25.0, random_state=42,class_weight="balanced")
    
elif(classifier_type == 'bagging'):
    model = BaggingClassifier(base_estimator=cart, n_estimators=200, max_samples=0.3)
    
elif(classifier_type == 'svm'):
    model = svm.SVC(C=10.0, kernel= "rbf", class_weight="balanced", gamma='auto') #rbf kernel
    
elif(classifier_type == 'randomforest'):
    model = RandomForestClassifier(n_estimators=400,class_weight="balanced",max_depth=5)

model.fit(all_features, labels)

## Predict the test features

In [ ]:
prediction = model.predict(all_feature_val)

In [ ]:
#######################
#Confustion Matrix
#######################
from sklearn.metrics import confusion_matrix
confusion_matrix(labels, prediction)

In [ ]:
#######################
#Accuracy
#######################
conf_mat = confusion_matrix(labels, prediction)
acc = (conf_mat[0][0] + conf_mat[1][1]) / len(labels)
acc

In [ ]:
#####################################################################
#Display precision, f1-score, senstivity and specificity
#####################################################################
print(classification_report(labels, prediction))

## Visualize the features using TSNE

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(meanstd_feature_val)
X_embedded.shape

In [ ]:
colors = ['r'] * 600 + ['b'] * 600

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(frameon=False)
plt.setp(ax, xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
plt.scatter(X_embedded[:, 0], X_embedded[:, 1],
        c=colors, marker="x")